In [3]:
import csv
import math
import re
from collections import defaultdict, namedtuple
from itertools import product
from types import SimpleNamespace

def is_integer(num):
    return int(num) == num

def sort_string(s):
    return ''.join(sorted(s))

# Day 1: Inverse captcha

In [17]:
def captcha(digits, step=1):
    size = len(digits)
    match_next = [
        int(v) if v == digits[(k+step) % size] else 0
        for k, v in enumerate(digits)
    ]
    return sum(match_next)

with open('inputs/day1', 'r') as f:
    digits = f.read().strip()
    
captcha(digits)

1119

In [18]:
captcha(digits, step=(len(digits)//2))

1420

# Day 2: Corruption Checksum

In [37]:
def min_max_diff(iterable):
    return max(iterable) - min(iterable)

def checksum(*args, row_func=min_max_diff):
    return sum([row_func(line) for line in args])

with open('inputs/day2', 'r') as f:
    reader = csv.reader(f, delimiter="\t")
    lines = [list(map(int, line)) for line in reader]

checksum(*lines)

51139

In [40]:
def evenly_divisible(iterable):
    divs = (p[0] / p[1] for p in product(iterable, repeat=2) if p[0] != p[1])
    for num in divs:
        if is_integer(num):
            return num
        
checksum(*lines, row_func=evenly_divisible)

272.0

# Day 3: Spiral Memory
First observation is that the lower right corner number of each ring in the spiral is equal to (2n+1)^2
Knowing the ring number is one part of the Manhattan Distance.  The second part of the distance is how far the we need to walk to the center of the edge. The coordinates of the center of each edge for a ring is given by: 
((2n+1)^2 - n, (2n+1)^2 - 3n, (2n+1)^2 - 5n, (2n+1)^2 - 7n)

In [3]:
def spiral_ring_radius(num):
    radius = math.ceil((math.sqrt(num) - 1)/2)
    return int(radius)

def spiral_ring_edge_centers(ring):
    corner = ((2 * ring) + 1) ** 2
    return [
        corner - 1 * ring,
        corner - 3 * ring,
        corner - 5 * ring,
        corner - 7 * ring,
    ]
    
def spiral_manhattan_distance(num):
    ring = spiral_ring_radius(num)
    centers = spiral_ring_edge_centers(ring)
    walks = [abs(num - c) for c in centers]
    return (ring, min(walks))

day3_input = 368078
spiral_manhattan_distance(day3_input)

(303, 68)

The new spiral values can be built with a recursive function.  
F(0) = 1
F(n) = F(n-1)

# Day 4: High-Entropy Passphrase

In [13]:
def is_valid_passphrase(passphrase):
    words = passphrase.strip().split(' ')
    return len(words) == len(set(words))
    
with open('inputs/day4', 'r') as f:
    lines = list(map(str.strip, f.readlines()))

sum([is_valid_passphrase(line) for line in lines])

466

In [18]:
def is_valid_passphrase_anagrams(passphrase):
    words = [sort_string(w) for w in passphrase.strip().split(' ')]
    return len(words) == len(set(words))

sum([is_valid_passphrase_anagrams(line) for line in lines])

251

# Day 5: A Maze of Twisty Trampolines, All Alike

In [36]:
def _simple_instr_change(i):
    return 1

def _complex_instr_change(i):
    return -1 if i >= 3 else 1
    
def jump_outside_stepcount(instructions, func=_simple_instr_change):
    curr = 0
    count = 0
    while True:
        try:
            moves = instructions[curr]
        except IndexError:
            break
            
        instructions[curr] += func(instructions[curr])
        curr += moves
        count += 1
        
    return count

with open('inputs/day5', 'r') as f:
    instructions = list(map(int, f.readlines()))
    
jump_outside_stepcount(instructions.copy())

387096

In [37]:
jump_outside_stepcount(instructions.copy(), func=_complex_instr_change)

28040648

# Day 6: Memory Reallocation

In [52]:
class Memory(list):
    def __str__(self):
        return ",".join(map(str, self))
        
    def reallocate(self):
        index = self.most_blocks
        size = len(self)
        blocks, self[index] = self[index], 0
        for i in range(1, blocks + 1):
            self[(index+i) % size] += 1

    @property
    def most_blocks(self):
        return self.index(max(self)) 

def memory_reallocation(memory):       
    history = [str(memory),]   
    while True:
        memory.reallocate()     
        if str(memory) in history:
            break
        else:
            history.append(str(memory))
            
    return len(history)

with open('inputs/day6', 'r') as f:
    memory_bank = Memory(map(int, f.read().split('\t')))
    
memory_reallocation(memory_bank)

11137

In [53]:
memory_reallocation(memory_bank)

1037

# Day 7: Recursive Circus
For part 2, we need to find the program node that has balanced children but is itself unbalanced

In [7]:
class ProgramTower(defaultdict):
    def get_bottom(self):
        for name, program in self.items():
            if not program.parent:
                return name

DAY7_RE = r'([a-z]+) \((\d+)\)(?: -\> (.*))?'

program_tower = ProgramTower(lambda: SimpleNamespace(weight=None, parent=None, supports=[]))
with open('inputs/day7', 'r') as f:
    for line in f:
        name, weight, supports = re.match(DAY7_RE, line).groups()
        
        program_tower[name].weight = weight
        if supports:
            program_tower[name].supports = list(map(str.strip, supports.split(',')))
        
        for support in program_tower[name].supports:
            program_tower[support].parent = name
            
program_tower.get_bottom()  


'gynfwly'